#Load Training Data

In [1]:
import os
import glob
import imageio
import numpy as np

# Input setup
# Prepare Data
dataset = "DIV2K_train_HR"
hr_image_filenames = os.listdir(dataset)  # print(hr_image_filenames)
hr_image_dir  = os.path.join(os.getcwd(), dataset)  # print(data_dir)
hr_image_data = glob.glob(os.path.join(hr_image_dir,"*.png")) # print(data)
dataset = "DIV2K_train_LR_bicubic\X4"
lr_image_filenames = os.listdir(dataset)  # print(hr_image_filenames)
lr_image_dir  = os.path.join(os.getcwd(), dataset)  # print(data_dir)
lr_image_data = glob.glob(os.path.join(lr_image_dir,"*.png")) # print(data)

hr_images = []
lr_images = []

print("# images:", len(hr_image_data))

for i in range(5): #range(len(data)):
    hr_image = imageio.imread(hr_image_data[i]).astype(np.float)
    hr_images.append(hr_image)
    lr_image = imageio.imread(lr_image_data[i]).astype(np.float)
    lr_images.append(lr_image)

print(hr_images[0].shape)
print(lr_images[0].shape)



# images: 800
(1404, 2040, 3)
(351, 510, 3)


#Define the Model

In [2]:
import tensorflow as tf
import tensorlayer as tfl

# Generator Model
def SRGAN_GEN(input_image):
    mu = 0
    sigma = 0.1
    # Layer 1: Convolution Input =. Output=.
    conv1_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev = sigma))
    conv1_bias   = tf.Variable(tf.zeros(64))
    conv1_output = tf.nn.conv2d(input_image, conv1_kernel, strides=[1, 1, 1, 1], act=tf.nn.relu, padding='SAME') + conv1_bias
    conv1_act = tf.nn.relu(conv1_output)
    tmp_conv_act = conv1_act
    # B residual blocks
    for i in range(16):
        # Layer: Convolution
        tmp_conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
        tmp_conv_bias   = tf.Variable(tf.zeros(64))
        tmp_conv_output = tf.nn.conv2d(tmp_conv_act, tmp_conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + tmp_conv_bias       
        # Layer: Batch Normalization
        tmp_conv_mean, tmp_conv_var = tf.nn.moments(tmp_conv_output, [0])
        tmp_conv_scale = tf.Variable(tf.ones([64]))
        tmp_conv_beta  = tf.Variable(tf.zeros([64]))
        tmp_conv_bn = tf.nn.batch_normalization(tmp_conv_output, tmp_conv_mean, tmp_conv_var, tmp_conv_beta, tmp_conv_scale, epsilon)        
        # Layer: Convolution
        tmp_conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
        tmp_conv_bias   = tf.Variable(tf.zeros(64))
        tmp_conv_output = tf.nn.conv2d(tmp_conv_bn, tmp_conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + tmp_conv_bias       
        # Layer: Batch Normalization
        tmp_conv_mean, tmp_conv_var = tf.nn.moments(tmp_conv_output, [0])
        tmp_conv_scale = tf.Variable(tf.ones([64]))
        tmp_conv_beta  = tf.Variable(tf.zeros([64]))
        tmp_conv_bn = tf.nn.batch_normalization(tmp_conv_output, tmp_conv_mean, tmp_conv_var, tmp_conv_beta, tmp_conv_scale, epsilon)        
        # Layer: Elementwise Add
        tmp_eltwise_out = tf.math.add([tmp_conv_act, tmp_conv_bn])
        tmp_conv_act = tmp_eltwise_out
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(tmp_conv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Batch Normalization
    conv_mean, conv_var = tf.nn.moments(conv_output, [0])
    conv_scale = tf.Variable(tf.ones([64]))
    conv_beta  = tf.Variable(tf.zeros([64]))
    conv_bn = tf.nn.batch_normalization(conv_output, conv_mean, conv_var, conv_beta, conv_scale, epsilon)        
    # Layer: Eltwise operations
    eltwise_out = tf.math.add([conv_bn, tmp_conv_act])
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(tmp_conv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Sub Pixel Convolution
    subpixconv_out = tfl.layers.SubpixelConv2d(conv_output, scale=2, n_out_channel=None, act=tf.nn.relu)
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(subpixconv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Sub Pixel Convolution
    subpixconv_out = tfl.layers.SubpixelConv2d(conv_output, scale=2, n_out_channel=None, act=tf.nn.relu)
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(subpixconv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    
    return conv_output



ImportError: No module named 'sklearn.__check_build._check_build'
___________________________________________________________________________
Contents of C:\Users\dwith\AppData\Roaming\Python\Python36\site-packages\sklearn\__check_build:
setup.py                  _check_build.cp36-win32.pyd__init__.py
__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.